In [ ]:
import sys

import numpy as np

sys.path.append('../../..')

from change_of_basis import cmat_of_tmat
from change_of_basis import tmat_of_cmat
from materials import get_materials_Cvec
from safe_module import closest
from utilities import print_matrix_matlab
from utilities import sm2v
from utilities import v2sm

In [ ]:
write = False

In [ ]:
Cvec_Brown, Cvec_Brown_stdv, *_ = get_materials_Cvec(material="Brown")

In [ ]:
Cmat_Brown = v2sm(Cvec_Brown)
Tmat_Brown = tmat_of_cmat(Cmat_Brown)

In [ ]:
def footer(filename, c_vec):
    
    with open(filename, 'a') as f:
        f.write('  if (iflag_aniso == IANISOTROPY_MODEL2) then\n')
        c = 0
        for i in range(1,7):
            for j in range(i,7):
                f.write(f'    c{i}{j} = {c_vec[c]:17.12f} * 1d9\n')
                c += 1
        f.write('  endif\n')
        f.write('\n')
        f.write('  end subroutine model_aniso')

In [ ]:
# Brown_SymGroup models

node_sequence    = ['TRIV','MONO','ORTH','TET','XISO','ISO']

Tmat_nodes = []
for sigma in node_sequence:
        print(f'adding node {sigma} ....')
        if sigma == "TRIV":
            Tmat_nodes.append(Tmat_Brown)
        else:
            Tmat_nodes.append(closest(Tmat_Brown, sigma))

In [ ]:
# Brown_SymGroups models

symmetry_classes = ['trivial', 'monoclinic', 'orthorhombic', 'tetragonal',
                    'transversely isotropic', 'isotropic']
models = ['TRIV', 'MONO_TRIV', 'MONO', 'ORTH_MONO', 'ORTH', 'TET_ORTH', 'TET',
          'XISO_TET', 'XISO', 'ISO_XISO', 'ISO']

for i, model in enumerate(models):
    
    if write:
        print(f'writing model {model} ....')
        filename = f'model_aniso_{model}.f90'
        ! cp model_aniso.f90 {filename}
    else:
        print(f'computing model {model} ....')
    
    if i%2==0:
        j = round(i/2)
        t_mat_test = Tmat_nodes[j]
        if j==0:
            textlines = ['! overwrite perturbation model 2 with the Brown2016'
                         ' map\n']
        else:
            textlines = [f'! overwrite perturbation model 2 with closest'
                         f' {symmetry_classes[j]} map to the Brown2016 map\n']
    else:
        j = round((i-1)/2)        
        t_mat1 = Tmat_nodes[j]
        t_mat2 = Tmat_nodes[j+1]
        t_mat_test = 0.5 * t_mat1 + 0.5 * t_mat2
        textlines = [f'! overwrite perturbation model 2 with map halfway'
                     f' between closest {symmetry_classes[j+1]} and'
                     f' {symmetry_classes[j]} map',
                     '! to the Brown2016 map\n']

    if write:
        with open(filename, 'a') as f:
            for textline in textlines:
                f.write(f'\n{textline}')
        
    c_mat_test = cmat_of_tmat(t_mat_test)   
    c_vec_test = sm2v(c_mat_test)
    
    if write:
        footer(filename, c_vec_test)    
        ! mv {filename} Brown_SymGroups/.
    
    print(f'printing model {model} ....')    
    print_matrix_matlab(c_mat_test)

In [ ]:
# Brown_t20 models

t_mat1 = Tmat_Brown
t_mat2 = closest(Tmat_Brown, 'ISO')

models_2 = ['160', '140', '120', '100', '80', '60', '40', '20', '00']
t_min    = -3/5
t_max    = 1
dt       = 1/5
npts     = round( (t_max - t_min) / dt + 1 )
frac     = np.linspace(t_min,t_max,npts)

# models_2 = ['160', '150', '140', '130', '120', '110', '100', '90', '80',
#             '70', '60', '50', '40', '30', '20', '10', '00']
# frac     = np.arange(-6,11) / 10

for i, model in enumerate(models_2):
    
    if write:
        print(f'writing model t{model} ....')
        filename = f'model_aniso_t{model}.f90'
        ! cp model_aniso.f90 {filename}
        
    if model == '00':
        textlines = ['! overwrite perturbation model 2 with closest isotropic'
                     ' map to the Brown2016 map\n']
    elif model == '100':
        textlines = ['! overwrite perturbation model 2 with the Brown2016'
                     'map\n']
    else:
        textlines = [f'! overwrite perturbation model 2 with map with {model}%'
                     ' anisotropy of Brown2016 map\n']
    
    if write:
        with open(filename, 'a') as f:
            for textline in textlines:
                f.write(f'\n{textline}')
        
    t_mat_test = (1-frac[i]) * t_mat1 + (frac[i]) * t_mat2
    c_mat_test = cmat_of_tmat(t_mat_test)
    c_vec_test = sm2v(c_mat_test)
        
    if write:
        footer(filename, c_vec_test)
        ! mv {filename} Brown_t20/.